# Running the _JWST_ Pipeline

### Environment set-up:

First, cd to the file folder that has the relevant uncal.fits file.

Then, in the tsch shell,

In [ ]:
conda create -n jwst_env python=3.11

Retrieving notices: ...working... done
Remove existing environment (y/[n])? 

In [ ]:
conda activate jwst_env

In [ ]:
pip3 install jwst

In [ ]:
pip3 install jupyter

Note: do not use _pip install jwst --upgrade_ -- instead, uninstall and reinstall to ensure all dependency packages are also upgraded.

### Setting environment variables for the Calibration References Data System (CRDS):

In a bash shell (activate using the command 'bash'), 

In [ ]:
mkdir $HOME/crds_cache
export CRDS_PATH=$HOME/crds_cache
export CRDS_SERVER_URL=https://jwst-crds.stsci.edu

Alternatively, the two environment variables can be set in tsch using:

In [ ]:
setenv CRDS_PATH $HOME/crds_cache
setenv CRDS_SERVER_URL https://jwst-crds.stsci.edu

Using the command 'printenv,' confirm that the two environment variables have been correctly set.  They must be defined before importing any _JWST_ or CRDS software packages.  If the CRDS environment variables were not set before initializing Python, they can be set in the Python session using 'os.environ': or CRDS software pacakges.

Additionally, do not modify the 'context' parameter to anything other than the default, as it is a .pars file that changes.

If the CRDS environment variables were not set before initializing Python, they can be set in the Python session using 'os.environ':

In [ ]:
import os
os.environ['CRDS_PATH'] = '$HOME/crds_cache'
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

### Stage 1: calwebb\_detector1:

This initial stage applies basic detector-level corrections to all exposure types, one exposure at a time.

To run the full pipeline, 

In [ ]:
python

import jwst
import crds
file_name = 'EXAMPLE.FITS'

from jwst.pipeline import Detector1Pipeline
result = Detector1Pipeline.call(file_name)

The first time this is run, the CRDS server should download all of the context/reference files needed for that run and place them in the CRDS_PATH-indicated directory.

It is not recommended to use the 'run' method to run a pipeline or step.  Instead, use the 'call' method.

BE PATIENT!

The 'pipeline' module contains all full pipeline stages.  For example, to import Stage 3 Imaging Processing requires the following command:

In [ ]:
from jwst.pipeline import Image3Pipeline

For individual pipeline steps, import by name from their respective module.  For example,

In [ ]:
from jwst.saturation import SaturationStep

Once imported, a pipeline or step is execute using the class's .call() method.  The input can be a string path to a file or an open DataModel object.  By default, no files are written out.

Additionally, by default, CRDS chooses the pipeline/step parameters and reference files based on information such as instrument, observing mode, date, etc.  Provided the current context was not changed from the default, these represent the best choice of parameters and reference files for the provided dataset.

To save the final pipeline output products to a file, include the 'save_results' parameter.  The 'result' definition is then:

In [ ]:
result = Detector1Pipeline.call(file_name, save_results=True)

#### In the command line:

The command line interface for individual steps and pipelines involves the 'strun' command.  Just as with the python environment, the CRDS environment variables must be defined before running any pipelines/steps.  Invoke 'strun,' then specify a pipeline or class name, then name the input file.  Either the pipeline class or alias name can be used ('jwst.pipeline.Detector1Pipeline' is the class which has the alias 'calwebb_detector1' and either name can be specified).  For example,

In [ ]:
$ strun calwebb_detector1 jw04501038001_03104_00001_nrca1_uncal.fits

An exit status of '0' indicates successful completion of the step or pipeline, while '1' means a general error has occured and '64' indicates that no science data was found.

Just like the python environment, 'save_results' can be specified:

In [ ]:
$ strun calwebb_detector1 jw04501038001_03104_00001_nrca1_uncal.fits
    --save_results=true

#### Logging Configuration

To save log information, first set up a configuration file to specify the desired level of logging messages.  This file must be in the same directory in which the pipeline is being run.

For example, if a file called 'pipeline_log.cfg' is created with the following contents,

In [ ]:
[*]
handler = file:pipeline.log
level = INFO

then log information will be written to a file named 'pipeline.log' with the level of detail set to 'INFO' (other options include 'DEBUG,' 'WARNING,' 'ERROR,' and 'CRITICAL').  Only messages at or above the specified level will be displayed.

When running the pipeline in a python environment, simply add the 'logcfg' parameter.  The 'result' definition from before is then:

In [ ]:
result = Detector1Pipeline.call(file_name, logcfg="pipeline-log.cfg")

In the command line, a similar option is added:

In [ ]:
$ strun calwebb_detector1 <input_file>
    --logcfg=pipeline-log.cfg